In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, Toplevel, Text, Canvas
from tkinter.filedialog import asksaveasfilename
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class QuadratDataApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Field Quadrat Data Analysis")
        self.root.configure(bg='lightgreen')
        self.setup_ui()
        self.fig = None
        self.colors = plt.cm.tab10.colors  # Define color palette
        
    def setup_ui(self):
        self.canvas = Canvas(self.root, bg='lightgreen')
        self.scrollable_frame = ttk.Frame(self.canvas)
        self.scrollable_frame.bind("<Configure>", lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))
        
        self.scrollable_window = self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        scrollbar_y = ttk.Scrollbar(self.root, orient="vertical", command=self.canvas.yview)
        scrollbar_x = ttk.Scrollbar(self.root, orient="horizontal", command=self.canvas.xview)
        self.canvas.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)
        
        scrollbar_y.pack(side="right", fill="y")
        scrollbar_x.pack(side="bottom", fill="x")
        self.canvas.pack(side="left", fill="both", expand=True)
        
        self.canvas.bind_all("<MouseWheel>", self._on_mousewheel)
        self.canvas.bind("<Button-4>", self._on_mousewheel)
        self.canvas.bind("<Button-5>", self._on_mousewheel)
        
        self.canvas.bind("<ButtonPress-1>", self._on_button_press)
        self.canvas.bind("<B1-Motion>", self._on_move_press)
        
        tk.Label(self.scrollable_frame, text="Number of Quadrats:", bg='lightgreen').grid(row=0, column=0)
        self.quadrat_count = tk.Entry(self.scrollable_frame)
        self.quadrat_count.grid(row=0, column=1)
        
        tk.Label(self.scrollable_frame, text="Number of Species:", bg='lightgreen').grid(row=1, column=0)
        self.species_count = tk.Entry(self.scrollable_frame)
        self.species_count.grid(row=1, column=1)
        
        tk.Button(self.scrollable_frame, text="Setup Table", command=self.setup_table).grid(row=2, column=0, columnspan=2)
        
        self.table_frame = tk.Frame(self.scrollable_frame, bg='lightgreen')
        self.table_frame.grid(row=3, column=0, columnspan=2)
        
        tk.Button(self.scrollable_frame, text="Calculate", command=self.calculate).grid(row=4, column=0)
        tk.Button(self.scrollable_frame, text="Generate Graphs", command=self.generate_graphs).grid(row=4, column=1)
        tk.Button(self.scrollable_frame, text="Download Data", command=self.download_data).grid(row=5, column=0, columnspan=2)
        
        self.graph_frame = tk.Frame(self.scrollable_frame, bg='lightgreen')
        self.graph_frame.grid(row=6, column=0, columnspan=2)
    
    def _on_mousewheel(self, event):
        self.canvas.yview_scroll(int(-1*(event.delta/120)), "units")
    
    def _on_button_press(self, event):
        self.canvas.scan_mark(event.x, event.y)
    
    def _on_move_press(self, event):
        self.canvas.scan_dragto(event.x, event.y, gain=1)
    
    def setup_table(self):
        for widget in self.table_frame.winfo_children():
            widget.destroy()
        
        try:
            self.quadrat_num = int(self.quadrat_count.get())
            self.species_num = int(self.species_count.get())
        except ValueError:
            messagebox.showerror("Input Error", "Please enter valid numbers for quadrats and species.")
            return
        
        self.data_entries = []
        for i in range(self.species_num + 1):
            row_entries = []
            for j in range(self.quadrat_num + 1):
                entry = tk.Entry(self.table_frame, width=10)
                entry.grid(row=i, column=j)
                if i == 0 and j > 0:
                    entry.insert(0, f"Quadrat {j}")
                    entry.config(state='readonly')
                elif j == 0 and i > 0:
                    entry.insert(0, f"Species {i}")
                    entry.config(state='readonly')
                row_entries.append(entry)
            self.data_entries.append(row_entries)
    
    def calculate(self):
        try:
            data = [[self.data_entries[i][j].get() for j in range(1, self.quadrat_num + 1)] for i in range(1, self.species_num + 1)]
            df = pd.DataFrame(data, columns=[f'Quadrat {i+1}' for i in range(self.quadrat_num)], index=[f'Species {i+1}' for i in range(self.species_num)])
            df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
            
            abundance = df.sum(axis=1)
            frequency = (df > 0).sum(axis=1)
            density = df.sum(axis=1) / self.quadrat_num
            relative_abundance = (abundance / abundance.sum()) * 100
            relative_frequency = (frequency / frequency.sum()) * 100
            relative_density = (density / density.sum()) * 100
            IVI = relative_abundance + relative_frequency + relative_density
            
            result = pd.DataFrame({
                'Abundance': abundance,
                'Frequency': frequency,
                'Density': density,
                'Relative Abundance': relative_abundance,
                'Relative Frequency': relative_frequency,
                'Relative Density': relative_density,
                'IVI': IVI
            })
            
            self.show_result_window(result)
            self.result = result
        except Exception as e:
            messagebox.showerror("Calculation Error", str(e))
    
    def show_result_window(self, result):
        result_window = Toplevel(self.root)
        result_window.title("Calculation Result")
        
        text_widget = Text(result_window, wrap='none')
        text_widget.insert(tk.END, result.to_string())
        text_widget.config(state='disabled')
        text_widget.pack(expand=True, fill='both')
    
    def generate_graphs(self):
        try:
            if self.fig:
                plt.close(self.fig)
            
            if self.graph_frame.winfo_children():
                for widget in self.graph_frame.winfo_children():
                    widget.destroy()

            fig, ax = plt.subplots(figsize=(8, 6))
            self.result[['Abundance', 'Frequency', 'Density', 'IVI']].plot(kind='bar', ax=ax, stacked=True, color=self.colors[:4])
            ax.set_xlabel('Species')
            ax.set_ylabel('Percentage')
            ax.set_title('Abundance to IVI')
            ax.legend(loc='upper right')

            canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
            canvas.draw()
            canvas.get_tk_widget().pack()

            button = tk.Button(self.graph_frame, text="Download Plot", command=self.download_individual_graph)
            button.pack()

            plt.tight_layout()
            self.fig = fig
        
        except AttributeError:
            messagebox.showerror("Graph Error", "Please perform calculations first.")
    
    def download_data(self):
        try:
            file_path = asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
            if file_path:
                self.result.to_csv(file_path, index=True)
                messagebox.showinfo("Download Successful", "Data has been successfully downloaded.")
        except Exception as e:
            messagebox.showerror("Download Error", str(e))
    
    def download_individual_graph(self):
        try:
            file_path = asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
            if file_path:
                fig = plt.figure(figsize=(6, 4))
                self.result[['Abundance', 'Frequency', 'Density', 'IVI']].plot(kind='bar', stacked=True, color=self.colors[:4])
                plt.title('Abundance to IVI')
                plt.xlabel('Species')
                plt.ylabel('Percentage')
                plt.legend(loc='upper right')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.savefig(file_path)
                plt.close(fig)
                messagebox.showinfo("Download Successful", "Graph has been successfully downloaded as PNG.")
        except Exception as e:
            messagebox.showerror("Download Error", str(e))

if __name__ == "__main__":
    root = tk.Tk()
    app = QuadratDataApp(root)
    root.mainloop()